In [23]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [24]:
# Selenium WebDriver 설정
url = "https://www.reddit.com/r/btc/"
driver = webdriver.Chrome()  # ChromeDriver 설치 필요
driver.get(url)

In [25]:
SCROLL_PAUSE_TIME = 10
MAX_SCROLLS = 100
scroll_count = 0
new_height = 0
last_height = 150

data = []

while scroll_count < MAX_SCROLLS:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        break
    
    last_height = new_height
    scroll_count += 1

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    posts = soup.find_all('article', class_='w-full m-0')
    
    for post in posts:
        shreddit_post = post.find('shreddit-post')
        
        author = shreddit_post.get('author') if shreddit_post else ''
        url = shreddit_post.get('content-href') if shreddit_post else ''
        title = shreddit_post.get('post-title') if shreddit_post else ''
        date = post.find('time')

        # 데이터 추가 (조건적으로 데이터가 None인지 확인)
        data.append({
            'author': author,
            'source': 'Reddit',
            'title': title,
            'url': url,
            'date': date.text if date else ''
        })

In [27]:
driver.quit()

In [26]:
# CSV 저장
df = pd.DataFrame(data)
file_path = r"C:\Users\jhkju\Documents\reddit_bitcoin3.csv"

df.to_csv(file_path, index=False)
print("CSV 파일로 저장 완료")

CSV 파일로 저장 완료
